  
# Trabajo Práctico 2

## Simulación de Eventos Discretos

#### Departamento de Computación - Facultad de Ciencias Exactas y Naturales - Universidad de Buenos Aires

25 de Octubre de 2021

**Grupo 04:** Confalonieri, Gisela - Lucero, Emiliano
    

----

# Modelo conceptual

En el presente trabajo modelaremos una simplificación del funcionamiento de la toma de radiografías.

Una radiografía es una técnica diagnóstica radiológica, en la que la imagen se obtiene al exponer al receptor de imagen radiográfica a una fuente de radiación de alta energía, comúnmente rayos X o radiación gamma procedente de isótopos radiactivos. Al interponer un objeto entre la fuente de radiación y el receptor, las partes más densas aparecen con diferentes tonos dentro de una escala de grises [1]. 

Los rayos X y los rayos gamma comprenden la porción de las altas energías y cortas longitudes de onda del espectro electromagnético. Los rayos gamma y los rayos X de igual longitud de onda tienen idénticas propiedades. Características de la radiación ionizante se basan en las siguientes características de los rayos X y de los rayos gamma:

* Tienen una longitud de onda inversamente proporcional a su energía
* No tienen carga eléctrica ni tienen masa.
* En el espacio, ambos **viajan en línea recta a la velocidad de la luz.**
* **Pueden penetrar la materia; la profundidad de la penetración depende de la longitud de onda de la radiación y de la naturaleza del material que es penetrado.**
* **Son absorbidos por la materia; el porcentaje de absorción es función de la densidad y el espesor del material y de la longitud de onda de la radiación.**
* Son dispersados por la materia; la cantidad de dispersión es función de la densidad de la materia y de la longitud de onda de la radiación.
* Pueden ionizar la materia.
* Pueden exponer un film/detector por ionización.
* Pueden producir fluorescencia en ciertos materiales.
* No pueden detectarse por medio de los sentidos humanos.
[1]


<!---Los rayos X, como los rayos de la luz visible, divergen desde su fuente de emisión y cubren mayores áreas en la medida que aumenta la distancia desde la fuente. NO VAMOS A CONSIDERAR ESTO EN PRINCIPIO--->


En la medida que los fotones de alta energía pasan a través de un material, su energía decrece. Esto se conoce como **atenuación**. La teoría de atenuación dice que fotones de mayores energías viajan a través de los tejidos más fácilmente que los fotones de bajas energías (esto se relaciona mucho con el efecto fotoeléctrico)[2]. <!---La probabilidad de absorción fotoeléctrica es aproximadamente proporcional a $$(Z/E)^3$$, donde Z es el número atómico del átomo de tejido y E es la energía del fotón. A medida que E es más grande, la probabilidad de interacción cae rápidamente .--->

<!--- lo que finalmente es la "placa de radiografía" es la marca de los rayos que han logrado atravesar las partes más blandas y llegar al chasis con la placa (por eso los huesos salen blancos y los pulmones, que tienen aire, negros). 

Además, los rayos se atenúan con la distancia, por lo que cambia la cantidad de radiación necesaria según la distancia del cañón al objetivo, y la calidad de imagen que se desea obtener.--->

Entonces, en este trabajo nos interesa modelar una vista "transversal" de este proceso: simulando el disparo de fotones desde ciertos puntos en un extremo de un plano y con cierta energía inicial, su avance en el espacio en línea recta, la atenuación de su energía en la medida que recorre el aire y/o se encuentra con materiales que puedan absorber parte de la energía, y la energía con la que llegan los fotones al extremo receptor del plano, representando la imagen radiológica resultante.

Por simplicidad, no consideraremos en nuestro primer modelo los aspectos de dispersión de radiación ni divergencia desde la fuente de emisión (es decir, que los rayos cubren mayores áreas en la medida que aumenta la distancia desde la fuente). Por lo tanto, nuestro modelo sólo considerará rayos que se disparan en línea recta desde la fuente emisora hacia el extremo receptor.

![](./imgs/idea_radiografia.png)

## Estructura del modelo


* Capa de transmisión.
* Capa de obstáculos.
* Ambas capas de la misma dimensión 2D.
* Un rayo por fila, atraviesa el medio de izquierda a derecha.
* Las celdas de la capa de obtáculos reprensentan el coeficiente de atenuación del obstaculo presente en ese lugar.
* Las celdas de la capa de transmisión representan la carga de los fotones ubicado en esa posición.
* Asumimos que los fotones se mueven con un ángulo recto.
* Asumimos que no hay rebotes de fotones.
* La primera columna de la capa de transmisión contiene las cargas iniciales de los fotones.
* La última columna de la capa de transmisión contiene las cargas finales de los fotones que alcanzaron el receptor.
* Las reglas definen el movimiento de los fotones a través del tiempo y la atenuación de las cargas a medida que se encuentran con obstaculos.

# Links - Referencias
[1] [Radiography - Wikipedia](https://en.wikipedia.org/wiki/Radiography)

[2] [What is X-Ray Attenuation](https://www.radiation-dosimetry.org/what-is-x-ray-attenuation-definition/)
* [X-Ray Mass Attenuation](https://www.physics.nist.gov/PhysRefData/XrayMassCoef/chap2.html)
* [X-Ray Mass Attenuation Coefficients](https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html)
* [Simulation of X-Ray NDT Imaging Techniques](https://www.ndt.net/article/wcndt00/papers/idn256/idn256.htm)

# Especificación

## Celda

$C=<X,Y,S,N,type,d,\tau, \delta_{int}, \delta_{ext},\lambda,ta>$

$X,Y,S = \{0,1,2\}$ 0 = Vacia, 1 = Ocupada por foton, 2 = Ocupada por Obstaculo

$N = \{(0, -1, 0), (0, 1, 0), (0, -1, 1), (0, 1, 1)\}$

$type = \text{transport}$

$d = vel_{propagacion}$

$\tau =$ definir las reglas primero

## Tablero

$GCTD=<X, Y, Xlist, Ylist, η, N, \{m, n\}, C, B, Z, select>$

$η = 4$. El de la izquierda y el de la derecha, en ambas capas.

$N = \{ (i_p, j_p) / \forall p \in N, p \in [1,η] \rightarrow i_p, j_p \in Z \wedge i_p,j_p \in \{-1, 1\} \}$

$B = \{C_{ij} | \forall (i = 1 \lor i = m \lor j = 1 \lor j = n) \wedge C_{ij} \in C\}$

## Fórmula Atenuación

La atenuación lineal esta dada por la fórmula:

$I (x) = I_0 e^{-ux}$

donde $I$ es la intensidad luego de la atenuación, $I_0$ es la intensidad entrante y $u$ es el coeficiente de atenuaciòn lineal.

Esta fórmula se va a usar en cada vez que en una celda ocurra una colisión entre un rayo y un obstáculo. La celda que representa al obstáculo guarda su coeficiente de atenuación.

# Implementación

## General

Este modelo consta de 2 tableros, uno con las intensidades de los fotones que se mueven en el espacio y otro con las
atenuaciones de los objetos que obstaculizan a los fotones

Los tableros son de la forma n x m y tienen la misma dimension. Los bordes son nowrapped y el retado es inercial (TODO: VER SI ESTA BIEN LO DEL RETARDO).

Los valores inciales del tablero de intensidades tiene la siguiente disposición:
* Las celdas de la primer columna tiene un valor mayor a 0, se los puede cargar mediante un archivo de entrada o con un generador
* Las celdas de la ultima columna tienen de valor -1, para señalizar que representan el output del modelo.
* Las celdas de las columnas intermedia tienen de valor 0.

Los valores iniciales del tablero de atenuaciones se determinan mediante la entrada provista.

Las celdas representan espacio de X cm. TODO: Ver bien el valor.

La velocidad de propagación de los fotones entre las celdas es constante y vale X. Si es muy bajo, lo normalizamos a una magnitud razonable para el simulador y luego lo corregimos. TODO: ver bien el valor.

### Vecindario

Utilizamos un vecindario simple, donde solo nos interesa los vecinos de la izquierda y de la derecha en ambos tableros.

```
neighbors : rayos(0, -1, 0) rayos(0, 0, 0) rayos(0, 1, 0)
neighbors : rayos(0, -1, 1) rayos(0, 0, 1) rayos(0, 1, 1)
```

## Reglas
* Regla avanzar sin obstaculos
    * Si mi valor en intensidades y en atenuaciones es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, tomo el valor de intensidad de mi vecino izquierdo.
```
    rule : {(0,-1,0)} vel_propagacion {(0,0,0) = 0 and (0,0,1) = 0 and (0, -1, 0) > 0}  
```
* Regla avanzar con obstaculos
    * Si mi valor en intensidades es 0 y en atenuaciones no es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, tomo el valor de intensidad de mi vecino izquierdo menos la atenuacion. (TODO: Hacer bien el calculo de atenuacion)
```
    rule : {(0,-1,0) * exp((0,0,1))} vel_propagacion {(0,0,0) = 0 and (0,0,1) > 0 and (0, -1, 0) > 0}  
```
* Regla borde derecho
    * En las celdas de la última columna, se mantiene el valor -1 mientras su vecino izquierdo no tenga valor distinto de 0.
```
    rule: {(0,0,0)} 0 {cellpos(1) = m and (0, -1, 0) = 0 and (0,0,0) = -1}
```
* Regla output
    * En las celdas de la última columna, si su vecino izquierdo tiene un valor de intensidad mayor a 0, se toma ese valor.
```
    rule: {(0,-1,0)} 0 {cellpos(1) = m and (0, -1, 0) > 0}
```
* Regla default
    * Es siempre verdadera. Mantiene el valor actual de la celda.
```
   rule : {(0,0,0)} 0 { t }
```

## Código

Inicializamos variables de entorno

In [77]:
from pathlib import Path

SED_HOME = Path.home().joinpath('SED')

#Directorio base donde está instalado el siumulador
CDPP_DIR = SED_HOME.joinpath('CDPP_ExtendedStates-codename-Santi')

CDPP_SRC = CDPP_DIR.joinpath('src')
CDPP_EXAMPLES = CDPP_DIR.joinpath('examples')
CDPP_SCRIPTS = CDPP_DIR.joinpath('scripts')
BASE_BIN = CDPP_SRC.joinpath('bin')
CDPP_ATOMICS = CDPP_SRC.joinpath('cd++/atomics')

CDPP_EXAMPLES_CELL = CDPP_EXAMPLES.joinpath('cell-devs')
CDPP_BIN = BASE_BIN.joinpath('cd++')
DRAWLOG_BIN = BASE_BIN.joinpath('drawlog')

Creamos la carpeta de nuestro proyecto en SED y copiamos los archivos correspondientes

In [78]:
%%bash -s "$CDPP_EXAMPLES"

cp -r rayos $1/

Mostramos rayos.ma

In [79]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat rayos.ma

[Top]
components : rayos

[rayos]
type : cell
dim : (10, 10, 2)
delay : transport
border : nowrapped
neighbors : rayos(0,-1,0)  rayos(0,0,0)  rayos(0,1,0)
neighbors : rayos(0,-1,1)  rayos(0,0,1)  rayos(0,1,1)
initialvalue : 1.0
initialcolvalue: 0 10
localtransition : rayos-rule
defaultDelayTime : 0.0

[rayos-rule]
% Regla avanzar sin obstaculos
% Si mi valor en intensidades y en atenuaciones es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, tomo el % valor de intensidad de mi vecino izquierdo.

rule : {(0,-1,0)} 0 {(0,0,0) = 0 and (0,0,1) = 0 and (0, -1, 0) > 0}  

% Regla avanzar con obstaculos
% Si mi valor en intensidades es 0 y en atenuaciones no es 0, y el valor de intensidad de mi vecino izquierdo es mayor a 0, % tomo el valor de intensidad de mi vecino izquierdo menos la atenuacion. (TODO: Hacer bien el calculo de atenuacion)

rule : {(0,-1,0) * exp((0,0,1))} 0 {(0,0,0) = 0 and (0,0,1) > 0 and (0, -1, 0) > 0}  

% Regla borde derecho
% En las celdas de la últim

Corremos la simulación

In [80]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

# Corre el modelo
../../src/bin/cd++ -m rayos.ma \
                   -l log.log \
                   -t 00:01:00:000

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from rayos.ma
Loading events from 
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: log.log
Output to: /dev/null
Tolerance set to: 1e-08
Configuration to show real numbers: Width = 12 - Precision = 5
Quantum:

Vemos los logs de la simulacion

In [81]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat log.log

[logfiles]
ParallelRoot : log.log00 
top : log.log202 
rayos : log.log01 
rayos(0,0,0) : log.log02 
rayos(0,0,1) : log.log03 
rayos(0,1,0) : log.log04 
rayos(0,1,1) : log.log05 
rayos(0,2,0) : log.log06 
rayos(0,2,1) : log.log07 
rayos(0,3,0) : log.log08 
rayos(0,3,1) : log.log09 
rayos(0,4,0) : log.log10 
rayos(0,4,1) : log.log11 
rayos(0,5,0) : log.log12 
rayos(0,5,1) : log.log13 
rayos(0,6,0) : log.log14 
rayos(0,6,1) : log.log15 
rayos(0,7,0) : log.log16 
rayos(0,7,1) : log.log17 
rayos(0,8,0) : log.log18 
rayos(0,8,1) : log.log19 
rayos(0,9,0) : log.log20 
rayos(0,9,1) : log.log21 
rayos(1,0,0) : log.log22 
rayos(1,0,1) : log.log23 
rayos(1,1,0) : log.log24 
rayos(1,1,1) : log.log25 
rayos(1,2,0) : log.log26 
rayos(1,2,1) : log.log27 
rayos(1,3,0) : log.log28 
rayos(1,3,1) : log.log29 
rayos(1,4,0) : log.log30 
rayos(1,4,1) : log.log31 
rayos(1,5,0) : log.log32 
rayos(1,5,1) : log.log33 
rayos(1,6,0) : log.log34 
rayos(1,6,1) : log.log35 
rayos(1,7,0) : log.log36 
rayos(1,7,1) : l

Veo el log de ParallelRoot

In [88]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat log.log00

0 / L / D / 00:00:00:000:0 / top(202) / 00:00:00:000:0 / ParallelRoot(00)
0 / L / D / 00:00:00:000:0 / top(202) / 00:00:00:000:0 / ParallelRoot(00)
0 / L / D / 00:00:00:000:0 / top(202) / ... / ParallelRoot(00)


Veo el log de rayos

In [85]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat log.log01

0 / L / I / 00:00:00:000:0 / top(202) / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,0,0)(02) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,0,1)(03) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,1,0)(04) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,1,1)(05) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,2,0)(06) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,2,1)(07) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,3,0)(08) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,3,1)(09) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,4,0)(10) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,4,1)(11) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,5,0)(12) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,5,1)(13) / 00:00:00:000:0 / rayos(01)
0 / L / D / 00:00:00:000:0 / rayos(0,6,0)(14) / 00

Veo el log de la primer celda de la primer fila

In [82]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat log.log02

0 / L / I / 00:00:00:000:0 / rayos(01) / rayos(0,0,0)(02)
0 / L / @ / 00:00:00:000:0 / rayos(01) / rayos(0,0,0)(02)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,0,0)(02)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,0,0)(02)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,0,0)(02)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,0,0)(02)
0 / L / * / 00:00:00:000:0 / rayos(01) / rayos(0,0,0)(02)


Veo el valor de la ultima celda de la primer fila

In [83]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/rayos/

cat log.log20

0 / L / I / 00:00:00:000:0 / rayos(01) / rayos(0,9,0)(20)
0 / L / @ / 00:00:00:000:0 / rayos(01) / rayos(0,9,0)(20)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,9,0)(20)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,9,0)(20)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,9,0)(20)
0 / L / X / 00:00:00:000:0 / rayos(01) / neighborchange /      1.00000 / rayos(0,9,0)(20)
0 / L / * / 00:00:00:000:0 / rayos(01) / rayos(0,9,0)(20)
